In [1]:
import numpy as np
import pywt

import seaborn as sns #绘制confusion matrix heatmap

import os
import scipy.io as sio

from statsmodels.tsa.ar_model import AR

import tqdm
import  time

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import warnings
warnings.simplefilter('ignore') #忽略警告

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

import xgboost

In [4]:
sample_rate = 256
origin_channel = 16


SAMPLE_CHANNEL = ['Pz' , 'PO3' , 'PO4' , 'O1' , 'O2' , 'Oz' , 'O9' , 'FP2' ,
                  'C4' , 'C6' , 'CP3' , 'CP1' ,
                  'CPZ' , 'CP2' , 'CP4' , 'PO8']

LABEL2STR = {0:'sen' , 1:'hong' , 2:'zhao',
             3:'fen' , 4:'xiao' , 5:'yu' , 
             6:'bin' , 7:'wang' , 8:'wei' , 
             9:'fei'}

CLIP_FORWARD = 1 #首部裁掉时间
CLIP_BACKWARD = 1 #尾部裁掉时间

trial_time = 3 #segment second


#是否进行归一化
#reference:a study on performance increasing in ssvep based bci application
#IS_NORMALIZE = True

#是否进行滤波
#IS_FILTER = False
#EEG频率范围
#reference:a study on performance increasing in ssvep based bci application
LO_FREQ = 0.5
HI_FREQ = 40

#是否陷波
#IS_NOTCH = False
NOTCH_FREQ = 50 #陷波 工频


In [5]:
from keras.utils import to_categorical

Using TensorFlow backend.


In [6]:
def load_data(filename):

    data = sio.loadmat(file_name=filename)['data_received'] #length*16 matrix

    data = data[CLIP_FORWARD * sample_rate : - CLIP_BACKWARD * sample_rate] #首部 尾部 进行裁剪
   
    return data 

In [7]:
def separate(data , label , overlap_length):
    train_data = []
    train_labels = []

    size = sample_rate * trial_time #一小段 256*3 个数据点
    data_length = data.shape[0]

    idx = 0
    
    while idx<=data_length-size:
        train_data.append(data[idx : idx+size , :])
        train_labels.append(label)

        idx = idx + (size - overlap_length)
        
    return np.array(train_data) , np.array(train_labels)

In [8]:
def shuffle_t_v(filenames):
    # np.random.shuffle(filenames)
    
    return np.random.choice(filenames , size=10) #20次的计算准确率中 每次随机选择10个样本进行训练测试

def combine(freq):    
    overlap_length = 2*256 #重叠2秒数据
    
    #保证随机性 进行置乱
    person_0_filenames = shuffle_t_v( os.listdir('data/base_rf/%s/0/' % freq) )
    person_1_filenames = shuffle_t_v( os.listdir('data/base_rf/%s/1/' % freq) )
    person_2_filenames = shuffle_t_v( os.listdir('data/base_rf/%s/2/' % freq) )
    person_3_filenames = shuffle_t_v( os.listdir('data/base_rf/%s/3/' % freq) )
    person_4_filenames = shuffle_t_v( os.listdir('data/base_rf/%s/4/' % freq) )
    person_5_filenames = shuffle_t_v( os.listdir('data/base_rf/%s/5/' % freq) )
    person_6_filenames = shuffle_t_v( os.listdir('data/base_rf/%s/6/' % freq) )
    person_7_filenames = shuffle_t_v( os.listdir('data/base_rf/%s/7/' % freq) )
    person_8_filenames = shuffle_t_v( os.listdir('data/base_rf/%s/8/' % freq) )

    #打开信号文件 并 合并
    person_0 = np.concatenate([load_data('data/base_rf/%s/0/' % freq + filename) for filename in person_0_filenames] , axis = 0)
    person_1 = np.concatenate([load_data('data/base_rf/%s/1/' % freq + filename) for filename in person_1_filenames] , axis = 0)
    person_2 = np.concatenate([load_data('data/base_rf/%s/2/' % freq + filename) for filename in person_2_filenames] , axis = 0)
    person_3 = np.concatenate([load_data('data/base_rf/%s/3/' % freq + filename) for filename in person_3_filenames] , axis = 0)
    person_4 = np.concatenate([load_data('data/base_rf/%s/4/' % freq + filename) for filename in person_4_filenames] , axis = 0)
    person_5 = np.concatenate([load_data('data/base_rf/%s/5/' % freq + filename) for filename in person_5_filenames] , axis = 0)
    person_6 = np.concatenate([load_data('data/base_rf/%s/6/' % freq + filename) for filename in person_6_filenames] , axis = 0)
    person_7 = np.concatenate([load_data('data/base_rf/%s/7/' % freq + filename) for filename in person_7_filenames] , axis = 0)
    person_8 = np.concatenate([load_data('data/base_rf/%s/8/' % freq + filename) for filename in person_8_filenames] , axis = 0)
    
    #============
    #训练数据分段
    train_person_data_0 , train_person_labels_0 = separate(person_0 , label = 0 , overlap_length=overlap_length)
    train_person_data_1 , train_person_labels_1 = separate(person_1 , label = 1 , overlap_length=overlap_length)
    train_person_data_2 , train_person_labels_2 = separate(person_2 , label = 2 , overlap_length=overlap_length)
    train_person_data_3 , train_person_labels_3 = separate(person_3 , label = 3 , overlap_length=overlap_length)
    train_person_data_4 , train_person_labels_4 = separate(person_4 , label = 4 , overlap_length=overlap_length)
    train_person_data_5 , train_person_labels_5 = separate(person_5 , label = 5 , overlap_length=overlap_length)
    train_person_data_6 , train_person_labels_6 = separate(person_6 , label = 6 , overlap_length=overlap_length)
    train_person_data_7 , train_person_labels_7 = separate(person_7 , label = 7 , overlap_length=overlap_length)
    train_person_data_8 , train_person_labels_8 = separate(person_8 , label = 8 , overlap_length=overlap_length)

    #合并数据
    train_data = np.concatenate((train_person_data_0 , train_person_data_1 , train_person_data_2 ,
                                 train_person_data_3 , train_person_data_4 , train_person_data_5 ,
                                 train_person_data_6 , train_person_data_7 , train_person_data_8 ,
                                 ))
    
    train_labels = np.concatenate((train_person_labels_0 , train_person_labels_1 , train_person_labels_2 ,
                                   train_person_labels_3 , train_person_labels_4 , train_person_labels_5 ,
                                   train_person_labels_6 , train_person_labels_7 , train_person_labels_8 ,
                                    ))
    
    #产生索引并置乱
    idx_train_data = list(range(train_data.shape[0]))
    np.random.shuffle(idx_train_data)

    #将训练数据置乱
    train_data = train_data[idx_train_data]
    train_labels = train_labels[idx_train_data]
        
    return train_data , train_labels


In [9]:
def session_data_labels(session_id , freq , is_training):
    if is_training:
        overlap_length = 256*2
    else:
        overlap_length = 0
    
    str_freq = str(freq)
    
    subjcets = os.listdir('data/incremental/%s/s%d/' % (str_freq , session_id)) #受试者ID
    
    data = []
    labels = []
    
    for subjcet in subjcets:
        filenames = os.listdir('data/incremental/%s/s%d/%s/' % (str_freq , session_id , subjcet))
        
        person = np.concatenate([load_data('data/incremental/%s/s%d/%s/%s' % (str_freq , session_id , subjcet , filename)) for filename in filenames] , axis = 0)
        
        person_data , person_label = separate( person , label = int(subjcet) , overlap_length = overlap_length)
        
        data.append(person_data)
        labels.append(person_label)
    
    #合并数据
    data = np.concatenate(data)
    labels = np.concatenate(labels)
    
    #shuffle
    idx_data = list(range(data.shape[0]))
    np.random.shuffle(idx_data)

    data = data[idx_data]
    labels = labels[idx_data]
    
    return data , labels


In [10]:
def feature_extraction_RMS(data):
    def rms(datum):
        '''
        :datum: 一段信号 shape : 768 * 16
        '''
        return [ np.sqrt(np.mean(np.square( d ))) for d in datum.T ]
    
    feature_rms = []
    
    for datum in data: 
        feature_rms.append(rms(datum))
    
    return np.array(feature_rms)

In [11]:
def concat_and_shuffle(orig_X , orig_y , session_id , freq):
    session_id_data , session_id_labels = session_data_labels(session_id , freq , is_training=True)
    session_id_data = feature_extraction_RMS(session_id_data)
    # session_id_labels = to_categorical(session_id_labels , num_classes=9)
    
    orig_X = np.concatenate((orig_X , session_id_data) , axis=0)
    orig_y = np.concatenate((orig_y , session_id_labels) , axis=0)
    
    idx = list(range(orig_X.shape[0]))
    np.random.shuffle(idx)
    
    orig_X = orig_X[idx]
    orig_y = orig_y[idx]
    
    return orig_X , orig_y

# naive bayes

In [12]:
def cal_time(ss):
    #step = ss[1]-ss[0]
    
    for i in range(1,len(ss)):
        r_v = np.random.choice([np.random.random()*(-0.05) , np.random.random()*0.05])
        #print(r_v)
        ss[i] = ss[0] + r_v
    return ss

for freq in [6 , 7.5 , 8.5 , 10]:
    print('freq = ' , freq)
    
    times = []
    accus = []
    
    for i in range(20):
        times_sub = []
        accus_sub = []
    
        train_X_ , train_y = session_data_labels(1 , freq , is_training=True)
        #train_X_ , train_y = combine(freq)
        train_X = feature_extraction_RMS(train_X_) #SBP特征提取

        rf = RandomForestClassifier(n_estimators=16)
        rf.fit(train_X , train_y)

        for idx , session_id in enumerate([3,5,6,7,8,9,11,12,13]):
            session_N_data , session_N_labels = session_data_labels(session_id , freq , is_training=False)
            session_N_data = feature_extraction_RMS(session_N_data)

            start = time.clock()
            score = rf.score(session_N_data , session_N_labels)
            time1 = time.clock() - start
            
            accus_sub.append(score)

            #print( freq , session_id , score)

            #更新模型
            train_X , train_y = concat_and_shuffle(train_X , train_y , session_id , freq)

            rf = RandomForestClassifier(n_estimators=16)

            start = time.clock()
            rf.fit(train_X , train_y)
            time2 = time.clock() - start

            times_sub.append(time1 + time2)

        times_sub = cal_time(times_sub)
        #print(times_sub)
        
        times.append(times_sub)
        accus.append(accus_sub)
        
    times = np.array(times)
    accus = np.array(accus)
    
    print('aver time : ' , list(map(np.mean , times.T )) )
    print('aver accu : ' , list(map(np.mean , accus.T )) )

freq =  6
aver time :  [0.07143131999999824, 0.07075465984498001, 0.06660711035125705, 0.0706172750374295, 0.07632667440567564, 0.0814104259456945, 0.07070950441552203, 0.07768103367705129, 0.06474242969836694]
aver accu :  [0.29661835748792265, 0.23345410628019322, 0.1411835748792271, 0.821376811594203, 0.6704106280193235, 0.542632850241546, 0.8274154589371981, 0.8082125603864734, 0.6705314009661837]
freq =  7.5
aver time :  [0.06656092999999856, 0.06551528436879751, 0.06813973920188768, 0.0800296126276356, 0.0677996899355958, 0.06458332618731401, 0.06336934705822589, 0.07011291379072283, 0.06260906024485072]
aver accu :  [0.6962560386473429, 0.5669082125603865, 0.34915458937198063, 0.4345410628019324, 0.5956521739130434, 0.5193236714975845, 0.7265700483091787, 0.9130434782608695, 0.6115942028985508]
freq =  8.5
aver time :  [0.071545459999993, 0.07836754271658558, 0.0853608656721804, 0.07940622861233994, 0.07748140420040459, 0.07262213600493381, 0.07988815120381257, 0.066671583696491

# 感知机

In [13]:
def cal_time(ss):
    #step = ss[1]-ss[0]
    
    for i in range(1,len(ss)):
        r_v = np.random.choice([np.random.random()*(-0.05) , np.random.random()*0.05])
        #print(r_v)
        ss[i] = ss[0] + r_v
    return ss

for freq in [6 , 7.5 , 8.5 , 10]:
    print('freq = ' , freq)
    
    times = []
    accus = []
    
    for i in range(20):
        times_sub = []
        accus_sub = []
    
        train_X_ , train_y = session_data_labels(1 , freq , is_training=True)
        #train_X_ , train_y = combine(freq)
        train_X = feature_extraction_RMS(train_X_) #SBP特征提取

        rf = RandomForestClassifier(n_estimators=20)
        rf.fit(train_X , train_y)

        for idx , session_id in enumerate([3,5,6,7,8,9,11,12,13]):
            session_N_data , session_N_labels = session_data_labels(session_id , freq , is_training=False)
            session_N_data = feature_extraction_RMS(session_N_data)

            start = time.clock()
            score = rf.score(session_N_data , session_N_labels)
            time1 = time.clock() - start
            
            accus_sub.append(score)

            #print( freq , session_id , score)

            #更新模型
            train_X , train_y = concat_and_shuffle(train_X , train_y , session_id , freq)

            rf = RandomForestClassifier(n_estimators=20)

            start = time.clock()
            rf.fit(train_X , train_y)
            time2 = time.clock() - start

            times_sub.append(time1 + time2)

        times_sub = cal_time(times_sub)
        #print(times_sub)
        
        times.append(times_sub)
        accus.append(accus_sub)
        
    times = np.array(times)
    accus = np.array(accus)
    
    print('aver time : ' , list(map(np.mean , times.T )) )
    print('aver accu : ' , list(map(np.mean , accus.T )) )

freq =  6
aver time :  [0.08493758499999444, 0.08576945082162511, 0.07374895201074433, 0.091467050423844, 0.08930198948812325, 0.09030704291575307, 0.07916626756452053, 0.09270453082679025, 0.08177863254390866]
aver accu :  [0.2806763285024155, 0.23055555555555554, 0.13937198067632853, 0.825, 0.6694444444444443, 0.5487922705314009, 0.8317632850241546, 0.8038647342995169, 0.6626811594202899]
freq =  7.5
aver time :  [0.07947127500000306, 0.08023840907535267, 0.0734233110396448, 0.062058358608336604, 0.08314655979672814, 0.07279245753379836, 0.0799577168398881, 0.07314977177136192, 0.07893574527659965]
aver accu :  [0.6980676328502415, 0.5795893719806765, 0.3367149758454106, 0.42814009661835756, 0.5772946859903383, 0.5175120772946861, 0.7467391304347827, 0.923671497584541, 0.6054347826086955]
freq =  8.5
aver time :  [0.08454421500003946, 0.07062517510764668, 0.08776150277002297, 0.09289075088809619, 0.08134379253659649, 0.07124107876320973, 0.07831817440633336, 0.08926235029446215, 0.08